In [1]:
import numpy as np
import random
import math
import matplotlib as mpl
import matplotlib.pyplot as plt
from scipy import stats

In [2]:
# Problem setup
n = 1000 #Size of each $X_1$ and $X_2$
theta = 0.3 #fraction of overlap (We need to estimate this)
string_size = int((2-theta)*n) #The length of $X$
string2_start = int((1-theta)*n) #Starting point of $X_2$
total_strings_generated = 10 #Number Of Strings $X$ we generate
num_of_masks = 50 #Number of lexicographic orderings we generate, The parameter u in the paper
postion_bit_truncation = 50 #Truncation of bit representation of starting location, parameter v in paper

In [3]:
# Generate sequences to estimate the rate-distortion pair later
string_set = []; #Array that holds $total_strings_generated$ number of strings of length $(2-\theta)n$
string1_set = []; #Array that holds first extracted of string of size n
string2_set = []; #Array that holds second extracted of string of size n
for i in range(total_strings_generated): #Creates random $Bern(1/2)$ strings $(X's)$ and extracts $X_1$ and $X_2$ from them
    string = np.random.binomial(size = string_size,n=1,p=0.5);
    string =''.join(map(str, string));
    string1 = string[0:n];
    string2 = string[string2_start:string_size];
    string_set.append(string);
    string1_set.append(string1);
    string2_set.append(string2);

In [4]:
# Function to Generate masks
def ObtainMasks(string_size,NumberOfMasks):
    mask = []
    for i in range(NumberOfMasks):
        temp_mask = np.random.binomial(size = string_size,n=1,p=0.5); #Just a binary string randomly generated
        temp_mask =''.join(map(str, temp_mask));
        mask.append(temp_mask)
    mask = np.array(mask)
    return mask 

1700

In [5]:
def xor(String1, String2):
    Output = ""
     
    # Loop to iterate over the
    # Binary Strings
    for i in range(min(len(String1),len(String2))):
         
        # If the Character matches
        if (String1[i] == String2[i]):
            Output += '0'
        else:
            Output += '1'
    return Output

In [6]:
# Function to return the decimal of the lexicographic first suffix string for a mask and string
def MaskBasedLexicographicFirst(String,Mask): #String and Mask are of same size
    suffixes = np.array([]) #Array that stores Suffixes of all these strings
    for i in range(len(String)): #Stores every suffix of String
        suffixes = np.append(suffixes,String[i:len(String)])
    masked_suffixes = np.array([])
    for i in range(len(String)):
        masked_suffixes= np.append(masked_suffixes,xor(suffixes[i],Mask)+"a"); # a is just used as a last priority addition
    masked_suffixes.sort();
    return(len(String) - len(masked_suffixes[0])+2)
    
        

In [7]:
# Function that truncates strings
def Truncation(string_size1,Position,BitTruncation): #Takes input as the Position, and the amount it is truncated (2^{BitTruncation would be the denominator})
        return (np.floor((Position/string_size1)*(2**(BitTruncation)))/2**(BitTruncation))

In [8]:
# function to obtain sketches based on location hashing
def ObtainLocationalSketches(String1,String2,string_size1,SketchSize,alpha_0): #Sketch size is B and alpha_0 = min(theta)/(2-min(theta))
    mask = ObtainMasks(len(String1),int(np.sqrt(SketchSize)));
    SketchesOne = np.array([]); #Arrays used to store sketches
    SketchesTwo = np.array([]);
    for i in range(len(mask)):
        positionone = MaskBasedLexicographicFirst(String1,mask[i]);
        positiontwo = MaskBasedLexicographicFirst(String2,mask[i]);
        positionone = Truncation(string_size1,positionone,int(np.sqrt(SketchSize)));
        positiontwo = Truncation(string_size1,positiontwo,int(np.sqrt(SketchSize)));
        SketchesOne = np.append(SketchesOne,positionone);
        SketchesTwo = np.append(SketchesTwo,positiontwo);
        return [SketchesOne,SketchesTwo]

In [47]:
A = "0000111100"

In [48]:
len(A)

10

In [49]:
string_1 = A[0:6]

In [50]:
len(string_1)

6

In [51]:
string_2 = A[4:10]

In [52]:
len(string_2)

6

In [141]:
temp = ObtainLocationalSketches(string1_set[0],string2_set[0],len(string1_set[0]),400,0)

In [142]:
len(string1_set[1])

1000

In [143]:
stats.mode(1-(temp[0]-temp[1]))[0][0]

0.5300006866455078

In [20]:
#Get rate distortion pair
def RateDistortionPair(theta,String1,String2,string_size1,SketchSize,alpha_0):
    temp = ObtainLocationalSketches(String1,String2,string_size1,SketchSize,alpha_0);
    temp1 = stats.mode(1-(temp[0]-temp[1]));
    theta_hat = temp1[0][0]
    Distortion = (theta-theta_hat)**2
    return([SketchSize,Distortion])

In [ ]:
Rate_DistortionPair_stringset = []
for j in range(total_strings_generated): #For each string
    Rate_DistortionPair_set = []
    for i in range(100,200): #Max rate B in the range 
        Rate_DistortionPair_set.append(RateDistortionPair(theta,string1_set[j],string2_set[j],n,i,0));
    Rate_DistortionPair_stringset.append(Rate_DistortionPair_set);
    

yo
yo
yo
yo
yo
yo
yo
yo
yo
yo
yo
yo
yo
yo
yo
yo
yo
yo
yo
yo
yo
yo
yo
yo
yo
yo
yo
yo
yo
yo
yo
yo
yo
yo
yo
yo
yo
yo
yo
yo
yo
yo
yo
yo
yo
yo
yo
yo
yo
yo
yo
yo
yo
yo
yo
yo
yo
yo
yo
yo
yo
yo
yo
yo
yo
yo
yo
yo
yo
yo
yo
yo
yo
yo
yo
yo
yo
yo
yo
yo
yo
yo
yo
yo
yo
yo
yo
yo
yo
yo
yo
yo
yo
yo
yo
yo
yo
yo
yo
yo
yo
yo
yo
yo
yo
yo
yo
yo
yo
yo
yo
yo
yo
yo
yo
yo
yo
yo
yo
yo
yo
yo
yo
yo
yo
yo
yo
yo
yo
yo
yo
yo
yo
yo
yo
yo
yo
yo
yo
yo
yo
yo
yo
yo
yo
yo
yo
yo
yo
yo
yo
yo
yo
yo
yo
yo
yo
yo
yo
yo
yo
yo
yo
yo
yo
yo
yo
yo
yo
yo
yo
yo
yo
yo
yo
yo
yo
yo
yo
yo
yo
yo
yo
yo
yo
yo
yo
yo
yo
yo
yo
yo
yo
yo
yo
yo
yo
yo
yo
yo
yo
yo
yo
yo
yo
yo
yo
yo
yo
yo
yo
yo
yo
yo
yo
yo
yo
yo
yo
yo
yo
yo
yo
yo
yo
yo
yo
yo
yo
yo
yo
yo
yo
yo
yo
yo
yo
yo
yo
yo
yo
yo
yo
yo
yo
yo
yo
yo
yo
yo
yo
yo
yo
yo
yo
yo
yo
yo
yo
yo
yo
yo
yo
yo
yo
yo
yo
yo
yo
yo
yo
yo
yo
yo
yo
yo
yo
yo
yo
yo
yo
yo
yo
yo
yo
yo
yo
yo
yo
yo
yo
yo
yo
yo
yo
yo
yo
yo
yo
yo
yo
yo
yo
yo
yo
yo
yo
yo
yo
yo
yo
yo
yo
yo
yo
yo
yo
yo
yo
yo
yo
yo
yo
yo
yo
yo
yo
yo
yo
yo
yo
yo
yo
y

In [ ]:
AvgRate_DistortionPair_set = np.mean(Rate_DistortionPair_stringset,axis=0)

yy = [x[1] for x in AvgRate_DistortionPair_set]

xx = [x[0] for x in AvgRate_DistortionPair_set]

fig, ax = plt.subplots()  # Create a figure containing a single axes.
ax.set_xlabel("D")
ax.set_ylabel("Rate")
ax.plot(yy,xx);  # Plot some data on the axes.